# **WEATHER FORECASTS FOR OUTDOOR CONCERT VENUES PROJECT**

## This project involves using Foursquare's API to get the location of popular outdoor concert venues across the U.S. and display them on a map, along with a 10-day thunderstorm forecast from AccuWeather's API

## Step 1: Call the Foursquare API and get name, city, state, postal code and lat/lon of the outdoor music venues

In [1]:
# import required libraries and install required programs 

import pandas as pd
import json
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium
import requests
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
!conda install -c conda-forge folium=0.5.0 --yes

print("Required libraries and programs imported and installed")

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Required libraries and programs imported and installed


In [2]:
# Define Foursquare credentials and version

CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180604'

In [3]:
# Since I cannot do a search query by country, in this case the US, I had to do some research on locations of outdoor concert venues. The 
# company LiveNation owns and operates dozens of venues across the US. Info can be found at https://www.livenation.com/venue-sitemap
# What I need to do is create a loop using city, state location of the more popular outdoor venues to create all of the
# Foursquare API request URLs. I manually created a csv file of the city, state to use to search for Amphitheatres near each city

# Create dataframe for cities
location = pd.read_csv (r'venue_list.csv')

# Store the results for each city
places = [] 

# Define variables for the API request URL 
search_query = 'Amphitheatre'
LIMIT = 15
radius = 75000
cities = location['City']

# Create for loop for the Foursquare API request URLs and return the results
for near in cities:
   url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&near={}&v={}&query={}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET,  
    near, 
    VERSION,
    search_query,
    radius, 
    LIMIT) 
   results = requests.get(url).json()
   places.append(results)

## Step 2: Use pertinent information from JSON to create a dataframe

In [4]:
# Create an empty dataframe for the information from the JSON

# Define the dataframe columns
column_name = ['Venue_Name', 'City', 'State', 'Zip_Code', 'Latitude', 'Longitude'] 

# Instantiate the dataframe and verify the columns are correct
venues = pd.DataFrame(columns=column_name)
venues

,Venue_Name,City,State,Zip_Code,Latitude,Longitude


In [5]:
# Create a loop to fill the dataframe one row at a time

for i in range(len(cities)):
    venue_data = places[i]['response']['venues'] # Get the relevant part of the JSON
    for data in venue_data:
        try:
            venue_name = data['name']
            city_name = data['location']['city']
            state_name = data['location']['state']
            zip_data = data['location']['postalCode']
            latitude = data['location']['lat']
            longitude = data['location']['lng']
            
            venues = venues.append({'Venue_Name': venue_name,
                                    'City': city_name,
                                    'State': state_name,
                                    'Zip_Code': zip_data,
                                    'Latitude': latitude,
                                    'Longitude': longitude,}, ignore_index=True)
        except KeyError:
            pass
    


In [6]:
# Check to see how many rows are in the dataframe and confirm there are 6 columns    
venues.shape

(256, 6)

In [7]:
# Look at the first five rows
venues.head()

,Venue_Name,City,State,Zip_Code,Latitude,Longitude
0,E. Lowell Rogers Amphitheatre,Glendale,AZ,85301,33.538792,-112.185368
1,Westside Amphitheatre,Phoenix,AZ,85035,33.472633,-112.234703
2,Westside Amphitheatre,Phoenix,AZ,85035,33.469850,-112.237898
3,Soleri Amphitheatre,Glendale,AZ,85302,33.567440,-112.188331
4,Amphitheatre Park,Gilbert,AZ,85296,33.344061,-111.699973


## Step 3: Clean the dataframe

In [8]:
# After reviewing the entire dataframe, there are duplicate venue names but located in different locations. 
# I don't want to remove the duplicates by name but rather by zip code. If there is a venue with the same name in the same
# zip code they are duplicates.

# Create a dataframe using .groupby to see which venues have the same name but are in a different location

venues_group = venues.groupby(['Venue_Name', 'Zip_Code']).count()
venues_group = venues_group.sort_values(by=['Zip_Code'])
venues_group

,,City,State,Latitude,Longitude
Venue_Name,Zip_Code,,,,
Green Hill Park Memorial Grove Amphitheatre,01605,2,2,2,2
Herter Amphitheatre,02134,2,2,2,2
Lesley University - Amphitheatre,02140,2,2,2,2
Vineyard Sound,02543,1,1,1,1
Province Lands Amphitheatre,02657,2,2,2,2
...,...,...,...,...,...
Becker Amphitheatre,98231,2,2,2,2
Centennial Amphitheater,98837,1,1,1,1
The Gorge Amphitheatre,98848,1,1,1,1


In [9]:
# There are several venues with the same name and in the same zip code. There are also several venues that are clearly not 
# outdoor concert venues--not sure how an AT&T Store showed up in a search for 'Amphitheatres'

# First, let's remove the duplicate venues from the same zip code then check to see if our removal was a success

venues = venues.drop_duplicates(subset='Zip_Code', keep='first')
venues_group = venues.groupby(['Venue_Name', 'Zip_Code']).count()
venues_group

,,City,State,Latitude,Longitude
Venue_Name,Zip_Code,,,,
AT&T Store,33130,1,1,1,1
Abacoa Amphitheatre,33458,1,1,1,1
Administration Building,21250,1,1,1,1
Alemeda County Fair Amphitheatre,94566,1,1,1,1
Ameris Bank Amphitheatre,30009,1,1,1,1
...,...,...,...,...,...
aurora amphitheatre,44202,1,1,1,1
charter amphitheatre,29681,1,1,1,1
pinecrest amphitheatre,30041,1,1,1,1


In [10]:
# Looks like we successfully removed the duplicates
# Now let's remove venues that are clearly not concert venues--basically any venue name that does not have the 
# word 'Amphitheatre'

venues = venues[venues['Venue_Name'].str.contains('Amphitheatre')].reset_index(drop=True)
venues

,Venue_Name,City,State,Zip_Code,Latitude,Longitude
0,E. Lowell Rogers Amphitheatre,Glendale,AZ,85301,33.538792,-112.185368
1,Westside Amphitheatre,Phoenix,AZ,85035,33.472633,-112.234703
2,Soleri Amphitheatre,Glendale,AZ,85302,33.567440,-112.188331
3,Amphitheatre Park,Gilbert,AZ,85296,33.344061,-111.699973
4,Mesa Amphitheatre,Mesa,AZ,85201,33.425680,-111.837444
...,...,...,...,...,...,...
114,Round Rock Amphitheatre,Round Rock,TX,78664,30.506010,-97.681377
115,Red Rocks Park & Amphitheatre,Morrison,CO,80465,39.665380,-105.205861
116,City Hall Amphitheatre,Denver,CO,80203,39.734710,-104.987195
117,Fiddler's Green Amphitheatre,Greenwood Village,CO,80111,39.600112,-104.893555


## Step 4: Plot the venue locations on a map

In [11]:
# Take the cleaned dataframe and plot venues on a map

# create map
map_venues = folium.Map(location=[39.828329, -98.579478], zoom_start=4)

# add markers to the map
markers_colors = []
for lat, lon, poi in zip(venues['Latitude'], venues['Longitude'], venues['Venue_Name']):
    label = folium.Popup('Venue:' + str(poi), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='black',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7).add_to(map_venues)
       
map_venues

## Step 5: Call AccuWeather locations API; use a loop with postal code to get the location key

In [12]:
# Use the zip codes from our venues dataframe to obtain the forecast location key for each location
        
# Store the results for each zip code
zip_code = [] 

# Define variable for the API request URL--we only need the zip code
postal_code = venues['Zip_Code']

# Create for loop for the Foursquare API request URLs and return the results
for q in postal_code:
   url = 'http://apidev.accuweather.com/locations/v1/search?q={}&apikey='.format(q)
   zips = requests.get(url).json()
   zip_code.append(zips)

zip_code[0]

[{'Version': 1,
  'Key': '36718_PC',
  'Type': 'PostalCode',
  'Rank': 35,
  'LocalizedName': 'Glendale',
  'EnglishName': 'Glendale',
  'PrimaryPostalCode': '85301',
  'Region': {'ID': 'NAM',
   'LocalizedName': 'North America',
   'EnglishName': 'North America'},
  'Country': {'ID': 'US',
   'LocalizedName': 'United States',
   'EnglishName': 'United States'},
  'AdministrativeArea': {'ID': 'AZ',
   'LocalizedName': 'Arizona',
   'EnglishName': 'Arizona',
   'Level': 1,
   'LocalizedType': 'State',
   'EnglishType': 'State',
   'CountryID': 'US'},
  'TimeZone': {'Code': 'MST',
   'Name': 'America/Phoenix',
   'GmtOffset': -7.0,
   'IsDaylightSaving': False,
   'NextOffsetChange': None},
  'GeoPosition': {'Latitude': 33.532,
   'Longitude': -112.178,
   'Elevation': {'Metric': {'Value': 333.0, 'Unit': 'm', 'UnitType': 5},
    'Imperial': {'Value': 1092.0, 'Unit': 'ft', 'UnitType': 0}}},
  'IsAlias': False,
  'ParentCity': {'Key': '331843',
   'LocalizedName': 'Glendale',
   'EnglishNa

In [13]:
# The locations API does return locations outside of the US but we can filter so we only get location keys for US locations

# Let's create a dataframe from the locations API that consists of location key and zip code
# We need to do a loop to fill the dataframe one row at a time

fc_df = pd.DataFrame() # Create an empty dataframe

for y in range(len(venues)):
    location_key = pd.json_normalize(zip_code[y])
    fc_df = fc_df.append(location_key, ignore_index = True)


In [14]:
# Let's take a look at the first 10 rows of our dataframe

fc_df.head(10)

,Version,Key,Type,Rank,LocalizedName,EnglishName,PrimaryPostalCode,IsAlias,SupplementalAdminAreas,DataSets,...,GeoPosition.Longitude,GeoPosition.Elevation.Metric.Value,GeoPosition.Elevation.Metric.Unit,GeoPosition.Elevation.Metric.UnitType,GeoPosition.Elevation.Imperial.Value,GeoPosition.Elevation.Imperial.Unit,GeoPosition.Elevation.Imperial.UnitType,ParentCity.Key,ParentCity.LocalizedName,ParentCity.EnglishName
0,1,36718_PC,PostalCode,35,Glendale,Glendale,85301,False,"[{'Level': 2, 'LocalizedName': 'Maricopa', 'En...","[AirQualityCurrentConditions, AirQualityForeca...",...,-112.178,333.0,m,5,1092.0,ft,0,331843,Glendale,Glendale
1,1,135236_PC,PostalCode,83,Schweitenkirchen,Schweitenkirchen,85301,False,"[{'Level': 2, 'LocalizedName': 'Pfaffenhofen a...","[AirQualityCurrentConditions, AirQualityForeca...",...,11.614,489.0,m,5,1603.0,ft,0,994110,Schweitenkirchen,Schweitenkirchen
2,1,185318_PC,PostalCode,500,Challans,Challans,85301,False,[],"[AirQualityCurrentConditions, AirQualityForeca...",...,-1.883,10.0,m,5,32.0,ft,0,NaN,NaN,NaN
3,1,36549_PC,PostalCode,21,Phoenix,Phoenix,85035,False,"[{'Level': 2, 'LocalizedName': 'Maricopa', 'En...","[AirQualityCurrentConditions, AirQualityForeca...",...,-112.191,311.0,m,5,1020.0,ft,0,346935,Phoenix,Phoenix
4,1,185157_PC,PostalCode,500,La Roche-sur-Yon,La Roche-sur-Yon,85035,False,[],"[AirQualityCurrentConditions, AirQualityForeca...",...,-1.433,65.0,m,5,212.0,ft,0,NaN,NaN,NaN
5,1,208209_PC,PostalCode,500,Noepoli,Noepoli,85035,False,[],"[AirQualityCurrentConditions, AirQualityForeca...",...,16.331,544.0,m,5,1785.0,ft,0,NaN,NaN,NaN
6,1,36719_PC,PostalCode,35,Glendale,Glendale,85302,False,"[{'Level': 2, 'LocalizedName': 'Maricopa', 'En...","[AirQualityCurrentConditions, AirQualityForeca...",...,-112.177,365.0,m,5,1198.0,ft,0,331843,Glendale,Glendale
7,1,135237_PC,PostalCode,83,Gerolsbach,Gerolsbach,85302,False,"[{'Level': 2, 'LocalizedName': 'Pfaffenhofen a...","[AirQualityCurrentConditions, AirQualityForeca...",...,11.356,499.0,m,5,1636.0,ft,0,172655,Gerolsbach,Gerolsbach
8,1,185319_PC,PostalCode,500,Challans,Challans,85302,False,[],"[AirQualityCurrentConditions, AirQualityForeca...",...,-1.883,10.0,m,5,32.0,ft,0,NaN,NaN,NaN
9,1,36714_PC,PostalCode,45,Gilbert,Gilbert,85296,False,"[{'Level': 2, 'LocalizedName': 'Maricopa', 'En...","[AirQualityCurrentConditions, AirQualityForeca...",...,-111.741,398.0,m,5,1306.0,ft,0,341804,Gilbert,Gilbert


In [15]:
# Let's take a look at the column names

fc_df.columns

Index(['Version', 'Key', 'Type', 'Rank', 'LocalizedName', 'EnglishName',
       'PrimaryPostalCode', 'IsAlias', 'SupplementalAdminAreas', 'DataSets',
       'Region.ID', 'Region.LocalizedName', 'Region.EnglishName', 'Country.ID',
       'Country.LocalizedName', 'Country.EnglishName', 'AdministrativeArea.ID',
       'AdministrativeArea.LocalizedName', 'AdministrativeArea.EnglishName',
       'AdministrativeArea.Level', 'AdministrativeArea.LocalizedType',
       'AdministrativeArea.EnglishType', 'AdministrativeArea.CountryID',
       'TimeZone.Code', 'TimeZone.Name', 'TimeZone.GmtOffset',
       'TimeZone.IsDaylightSaving', 'TimeZone.NextOffsetChange',
       'GeoPosition.Latitude', 'GeoPosition.Longitude',
       'GeoPosition.Elevation.Metric.Value',
       'GeoPosition.Elevation.Metric.Unit',
       'GeoPosition.Elevation.Metric.UnitType',
       'GeoPosition.Elevation.Imperial.Value',
       'GeoPosition.Elevation.Imperial.Unit',
       'GeoPosition.Elevation.Imperial.UnitType', 'Pare

In [16]:
# Only interested in creating a dataframe consisting of forecast location key ('Key'), postal code ('PrimaryPostalCode') and 
# country code ('Country.ID')
# Thus, we will clean the dataframe to include those 3 columns

fc_df = fc_df[['Key', 'PrimaryPostalCode', 'Country.ID']]
fc_df.columns = ['Key', 'PrimaryPostalCode', 'Country']
fc_df

,Key,PrimaryPostalCode,Country
0,36718_PC,85301,US
1,135236_PC,85301,DE
2,185318_PC,85301,FR
3,36549_PC,85035,US
4,185157_PC,85035,FR
...,...,...,...
734,35083_PC,80203,US
735,183204_PC,80203,FR
736,35047_PC,80111,US
737,134701_PC,80538,DE


In [17]:
# Only include rows where Country = 'US'
fc_df = fc_df[fc_df.Country == 'US']
fc_df = fc_df.reset_index(drop=True)
fc_df

,Key,PrimaryPostalCode,Country
0,36718_PC,85301,US
1,36549_PC,85035,US
2,36719_PC,85302,US
3,36714_PC,85296,US
4,36626_PC,85201,US
...,...,...,...
113,34424_PC,78664,US
114,35208_PC,80465,US
115,35083_PC,80203,US
116,35047_PC,80111,US


## Step 6: Call AccuWeather forecast API; use a loop with location key to get the 10-day weather forecast

In [18]:
# Now that we have the required location key for each zip code where a venue is located, we can create a loop to
# call the AccuWeather API and obtain the 10-day forecasts for each location

# Store the results of API call to get the forecasts
forecast = []

# In the API url, we only need the location key
# Create a loop to build a url for each API call and append the results to a file
# In Step 7 below we will convert the file to a dataframe

# Define variable for location key
key = fc_df['Key']

for locationKey in key:
   url = 'http://api.accuweather.com/forecasts/v1/daily/10day/{}?apikey=16803'.format(locationKey)
   tendayfc = requests.get(url).json()
   forecast.append(tendayfc)

# Check the first entry of json file

forecast[0]

{'Headline': {'EffectiveDate': '2020-08-06T13:00:00-07:00',
  'EffectiveEpochDate': 1596744000,
  'Severity': 7,
  'Text': 'Very warm from this afternoon to Saturday afternoon',
  'Category': 'heat',
  'EndDate': '2020-08-08T19:00:00-07:00',
  'EndEpochDate': 1596938400,
  'MobileLink': 'http://m.accuweather.com/en/us/glendale-az/85301/extended-weather-forecast/36718_pc?lang=en-us',
  'Link': 'http://www.accuweather.com/en/us/glendale-az/85301/daily-weather-forecast/36718_pc?lang=en-us'},
 'DailyForecasts': [{'Date': '2020-08-06T07:00:00-07:00',
   'EpochDate': 1596722400,
   'Temperature': {'Minimum': {'Value': 82.0, 'Unit': 'F', 'UnitType': 18},
    'Maximum': {'Value': 107.0, 'Unit': 'F', 'UnitType': 18}},
   'Day': {'Icon': 1, 'IconPhrase': 'Sunny', 'HasPrecipitation': False},
   'Night': {'Icon': 34,
    'IconPhrase': 'Mostly clear',
    'HasPrecipitation': False},
   'Sources': ['AccuWeather'],
   'MobileLink': 'http://m.accuweather.com/en/us/glendale-az/85301/daily-weather-forec

## Step 7: Convert venue forecasts from json to dataframe

In [19]:
# Let's create a dataframe from the forecast API for each venue
# We need to do a loop to fill the dataframe one row at a time
# We want everything included in 'DailyForecasts' but there are 10 days of forecasts

fc_venue = pd.DataFrame() # Create an empty dataframe

for a in range(len(fc_df)):
    for b in range(10):
        fc_tenday = pd.json_normalize(forecast[a]['DailyForecasts'][b])
        fc_venue = fc_venue.append(fc_tenday, ignore_index = True)


In [20]:
# Let's look at the first 20 rows of the dataframe. There should be a 10-day forecast for each venue.
fc_venue.head(20)

,Date,EpochDate,Sources,MobileLink,Link,Temperature.Minimum.Value,Temperature.Minimum.Unit,Temperature.Minimum.UnitType,Temperature.Maximum.Value,Temperature.Maximum.Unit,...,Day.Icon,Day.IconPhrase,Day.HasPrecipitation,Night.Icon,Night.IconPhrase,Night.HasPrecipitation,Day.PrecipitationType,Day.PrecipitationIntensity,Night.PrecipitationType,Night.PrecipitationIntensity
0,2020-08-06T07:00:00-07:00,1596722400,[AccuWeather],http://m.accuweather.com/en/us/glendale-az/853...,http://www.accuweather.com/en/us/glendale-az/8...,82.0,F,18,107.0,F,...,1,Sunny,False,34,Mostly clear,False,NaN,NaN,NaN,NaN
1,2020-08-07T07:00:00-07:00,1596808800,[AccuWeather],http://m.accuweather.com/en/us/glendale-az/853...,http://www.accuweather.com/en/us/glendale-az/8...,82.0,F,18,107.0,F,...,1,Sunny,False,33,Clear,False,NaN,NaN,NaN,NaN
2,2020-08-08T07:00:00-07:00,1596895200,[AccuWeather],http://m.accuweather.com/en/us/glendale-az/853...,http://www.accuweather.com/en/us/glendale-az/8...,84.0,F,18,109.0,F,...,1,Sunny,False,33,Clear,False,NaN,NaN,NaN,NaN
3,2020-08-09T07:00:00-07:00,1596981600,[AccuWeather],http://m.accuweather.com/en/us/glendale-az/853...,http://www.accuweather.com/en/us/glendale-az/8...,84.0,F,18,110.0,F,...,1,Sunny,False,34,Mostly clear,False,NaN,NaN,NaN,NaN
4,2020-08-10T07:00:00-07:00,1597068000,[AccuWeather],http://m.accuweather.com/en/us/glendale-az/853...,http://www.accuweather.com/en/us/glendale-az/8...,84.0,F,18,110.0,F,...,2,Mostly sunny,False,34,Mostly clear,False,NaN,NaN,NaN,NaN
5,2020-08-11T07:00:00-07:00,1597154400,[AccuWeather],http://m.accuweather.com/en/us/glendale-az/853...,http://www.accuweather.com/en/us/glendale-az/8...,89.0,F,18,110.0,F,...,1,Sunny,False,34,Mostly clear,False,NaN,NaN,NaN,NaN
6,2020-08-12T07:00:00-07:00,1597240800,[AccuWeather],http://m.accuweather.com/en/us/glendale-az/853...,http://www.accuweather.com/en/us/glendale-az/8...,87.0,F,18,110.0,F,...,2,Mostly sunny,False,34,Mostly clear,False,NaN,NaN,NaN,NaN
7,2020-08-13T07:00:00-07:00,1597327200,[AccuWeather],http://m.accuweather.com/en/us/glendale-az/853...,http://www.accuweather.com/en/us/glendale-az/8...,82.0,F,18,105.0,F,...,2,Mostly sunny,False,34,Mostly clear,False,NaN,NaN,NaN,NaN
8,2020-08-14T07:00:00-07:00,1597413600,[AccuWeather],http://m.accuweather.com/en/us/glendale-az/853...,http://www.accuweather.com/en/us/glendale-az/8...,84.0,F,18,110.0,F,...,3,Partly sunny,False,34,Mostly clear,False,NaN,NaN,NaN,NaN
9,2020-08-15T07:00:00-07:00,1597500000,[AccuWeather],http://m.accuweather.com/en/us/glendale-az/853...,http://www.accuweather.com/en/us/glendale-az/8...,85.0,F,18,111.0,F,...,2,Mostly sunny,False,34,Mostly clear,False,NaN,NaN,NaN,NaN


In [21]:
# Since there are 118 venues and a 10-day forecast for each venue there should be 1180 rows in our dataframe...let's confirm.
fc_venue.shape

(1180, 21)

## Step 8: Clean the venue forecast dataframe

In [22]:
# There are 21 columns in the dataframe but we will not need them all. Let's look at the column names.
fc_venue.columns

Index(['Date', 'EpochDate', 'Sources', 'MobileLink', 'Link',
       'Temperature.Minimum.Value', 'Temperature.Minimum.Unit',
       'Temperature.Minimum.UnitType', 'Temperature.Maximum.Value',
       'Temperature.Maximum.Unit', 'Temperature.Maximum.UnitType', 'Day.Icon',
       'Day.IconPhrase', 'Day.HasPrecipitation', 'Night.Icon',
       'Night.IconPhrase', 'Night.HasPrecipitation', 'Day.PrecipitationType',
       'Day.PrecipitationIntensity', 'Night.PrecipitationType',
       'Night.PrecipitationIntensity'],
      dtype='object')

In [23]:
# Let's have only year, month, day in the 'Date' column
fc_venue['Date'] = [x[:10] for x in fc_venue['Date']]
fc_venue

,Date,EpochDate,Sources,MobileLink,Link,Temperature.Minimum.Value,Temperature.Minimum.Unit,Temperature.Minimum.UnitType,Temperature.Maximum.Value,Temperature.Maximum.Unit,...,Day.Icon,Day.IconPhrase,Day.HasPrecipitation,Night.Icon,Night.IconPhrase,Night.HasPrecipitation,Day.PrecipitationType,Day.PrecipitationIntensity,Night.PrecipitationType,Night.PrecipitationIntensity
0,2020-08-06,1596722400,[AccuWeather],http://m.accuweather.com/en/us/glendale-az/853...,http://www.accuweather.com/en/us/glendale-az/8...,82.0,F,18,107.0,F,...,1,Sunny,False,34,Mostly clear,False,NaN,NaN,NaN,NaN
1,2020-08-07,1596808800,[AccuWeather],http://m.accuweather.com/en/us/glendale-az/853...,http://www.accuweather.com/en/us/glendale-az/8...,82.0,F,18,107.0,F,...,1,Sunny,False,33,Clear,False,NaN,NaN,NaN,NaN
2,2020-08-08,1596895200,[AccuWeather],http://m.accuweather.com/en/us/glendale-az/853...,http://www.accuweather.com/en/us/glendale-az/8...,84.0,F,18,109.0,F,...,1,Sunny,False,33,Clear,False,NaN,NaN,NaN,NaN
3,2020-08-09,1596981600,[AccuWeather],http://m.accuweather.com/en/us/glendale-az/853...,http://www.accuweather.com/en/us/glendale-az/8...,84.0,F,18,110.0,F,...,1,Sunny,False,34,Mostly clear,False,NaN,NaN,NaN,NaN
4,2020-08-10,1597068000,[AccuWeather],http://m.accuweather.com/en/us/glendale-az/853...,http://www.accuweather.com/en/us/glendale-az/8...,84.0,F,18,110.0,F,...,2,Mostly sunny,False,34,Mostly clear,False,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1175,2020-08-11,1597150800,[AccuWeather],http://m.accuweather.com/en/us/loveland-co/805...,http://www.accuweather.com/en/us/loveland-co/8...,59.0,F,18,93.0,F,...,4,Intermittent clouds,False,34,Mostly clear,False,NaN,NaN,NaN,NaN
1176,2020-08-12,1597237200,[AccuWeather],http://m.accuweather.com/en/us/loveland-co/805...,http://www.accuweather.com/en/us/loveland-co/8...,59.0,F,18,93.0,F,...,3,Partly sunny,False,34,Mostly clear,False,NaN,NaN,NaN,NaN
1177,2020-08-13,1597323600,[AccuWeather],http://m.accuweather.com/en/us/loveland-co/805...,http://www.accuweather.com/en/us/loveland-co/8...,57.0,F,18,90.0,F,...,3,Partly sunny,False,33,Clear,False,NaN,NaN,NaN,NaN
1178,2020-08-14,1597410000,[AccuWeather],http://m.accuweather.com/en/us/loveland-co/805...,http://www.accuweather.com/en/us/loveland-co/8...,55.0,F,18,85.0,F,...,4,Intermittent clouds,False,34,Mostly clear,False,NaN,NaN,NaN,NaN


In [24]:
# We don't need anything related to temperature, only precipitation

fc_venue = fc_venue[['Date', 'Link', 'Day.Icon', 'Day.HasPrecipitation', 'Night.Icon', 'Night.HasPrecipitation',
                    'Day.PrecipitationIntensity', 'Night.PrecipitationIntensity']]

# Verify the changes
fc_venue

,Date,Link,Day.Icon,Day.HasPrecipitation,Night.Icon,Night.HasPrecipitation,Day.PrecipitationIntensity,Night.PrecipitationIntensity
0,2020-08-06,http://www.accuweather.com/en/us/glendale-az/8...,1,False,34,False,NaN,NaN
1,2020-08-07,http://www.accuweather.com/en/us/glendale-az/8...,1,False,33,False,NaN,NaN
2,2020-08-08,http://www.accuweather.com/en/us/glendale-az/8...,1,False,33,False,NaN,NaN
3,2020-08-09,http://www.accuweather.com/en/us/glendale-az/8...,1,False,34,False,NaN,NaN
4,2020-08-10,http://www.accuweather.com/en/us/glendale-az/8...,2,False,34,False,NaN,NaN
...,...,...,...,...,...,...,...,...
1175,2020-08-11,http://www.accuweather.com/en/us/loveland-co/8...,4,False,34,False,NaN,NaN
1176,2020-08-12,http://www.accuweather.com/en/us/loveland-co/8...,3,False,34,False,NaN,NaN
1177,2020-08-13,http://www.accuweather.com/en/us/loveland-co/8...,3,False,33,False,NaN,NaN
1178,2020-08-14,http://www.accuweather.com/en/us/loveland-co/8...,4,False,34,False,NaN,NaN


In [25]:
# Change the names of columns that contain a '.'
fc_venue.columns = ['Date', 'Link', 'Day_Icon', 'Day_HasPrecip', 'Night_Icon', 'Night_HasPrecip', 'Day_Intensity',
                   'Night_Intensity']

fc_venue

,Date,Link,Day_Icon,Day_HasPrecip,Night_Icon,Night_HasPrecip,Day_Intensity,Night_Intensity
0,2020-08-06,http://www.accuweather.com/en/us/glendale-az/8...,1,False,34,False,NaN,NaN
1,2020-08-07,http://www.accuweather.com/en/us/glendale-az/8...,1,False,33,False,NaN,NaN
2,2020-08-08,http://www.accuweather.com/en/us/glendale-az/8...,1,False,33,False,NaN,NaN
3,2020-08-09,http://www.accuweather.com/en/us/glendale-az/8...,1,False,34,False,NaN,NaN
4,2020-08-10,http://www.accuweather.com/en/us/glendale-az/8...,2,False,34,False,NaN,NaN
...,...,...,...,...,...,...,...,...
1175,2020-08-11,http://www.accuweather.com/en/us/loveland-co/8...,4,False,34,False,NaN,NaN
1176,2020-08-12,http://www.accuweather.com/en/us/loveland-co/8...,3,False,34,False,NaN,NaN
1177,2020-08-13,http://www.accuweather.com/en/us/loveland-co/8...,3,False,33,False,NaN,NaN
1178,2020-08-14,http://www.accuweather.com/en/us/loveland-co/8...,4,False,34,False,NaN,NaN


In [26]:
# We are going to color code the forecasts (green = no thunderstorms, yellow = thunderstorms-moderate intensity,
# red = thunderstorms-heavy intensity)

# Let's assign 'Yes' or 'No' in a new columns in fc_venue called 'Thunder'. 
# This will tell us whether thunderstorms are in the forecast.

# Create a function for the day and night icons to return 'Yes' or 'No' based on the icon number
def d(row):
    if row['Day_Icon'] == 15:
        vald = 'Yes'
    elif row['Day_Icon'] == 16:
        vald = 'Yes'
    elif row['Day_Icon'] == 17:
        vald = 'Yes'
    elif row['Night_Icon'] == 41:
        vald = 'Yes'
    elif row['Night_Icon'] == 42:
        vald = 'Yes'
    else:
        vald = 'No'
    return vald

# Add Thunder to fc_venue dataframe
fc_venue['Thunder'] = fc_venue.apply(d, axis=1)

fc_venue

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Date,Link,Day_Icon,Day_HasPrecip,Night_Icon,Night_HasPrecip,Day_Intensity,Night_Intensity,Thunder
0,2020-08-06,http://www.accuweather.com/en/us/glendale-az/8...,1,False,34,False,NaN,NaN,No
1,2020-08-07,http://www.accuweather.com/en/us/glendale-az/8...,1,False,33,False,NaN,NaN,No
2,2020-08-08,http://www.accuweather.com/en/us/glendale-az/8...,1,False,33,False,NaN,NaN,No
3,2020-08-09,http://www.accuweather.com/en/us/glendale-az/8...,1,False,34,False,NaN,NaN,No
4,2020-08-10,http://www.accuweather.com/en/us/glendale-az/8...,2,False,34,False,NaN,NaN,No
...,...,...,...,...,...,...,...,...,...
1175,2020-08-11,http://www.accuweather.com/en/us/loveland-co/8...,4,False,34,False,NaN,NaN,No
1176,2020-08-12,http://www.accuweather.com/en/us/loveland-co/8...,3,False,34,False,NaN,NaN,No
1177,2020-08-13,http://www.accuweather.com/en/us/loveland-co/8...,3,False,33,False,NaN,NaN,No
1178,2020-08-14,http://www.accuweather.com/en/us/loveland-co/8...,4,False,34,False,NaN,NaN,No


In [27]:
# The possible values precipitation intensity for Day_Intensity and Night_Intensity are 'NaN' if dry, 'Light', 'Moderate'
# and 'Heavy'. Let's create a column called 'Intensity' and assign a number from 0 to 3 based on the forecast intensity.

# Define our function
def t(row):
    if row['Day_Intensity'] == 'Light':
        valt = 1
    elif row['Day_Intensity'] == 'Moderate':
        valt = 2
    elif row['Day_Intensity'] == 'Heavy':
        valt = 3
    elif row['Night_Intensity'] == 'Light':
        valt = 1
    elif row['Night_Intensity'] == 'Moderate':
        valt = 2
    elif row['Night_Intensity'] == 'Heavy':
        valt = 3
    else:
        valt = 0
    return valt

# Add 'Intensity' column and fill rows with the values from the function
fc_venue['Intensity'] = fc_venue.apply(t, axis=1)

fc_venue

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Date,Link,Day_Icon,Day_HasPrecip,Night_Icon,Night_HasPrecip,Day_Intensity,Night_Intensity,Thunder,Intensity
0,2020-08-06,http://www.accuweather.com/en/us/glendale-az/8...,1,False,34,False,NaN,NaN,No,0
1,2020-08-07,http://www.accuweather.com/en/us/glendale-az/8...,1,False,33,False,NaN,NaN,No,0
2,2020-08-08,http://www.accuweather.com/en/us/glendale-az/8...,1,False,33,False,NaN,NaN,No,0
3,2020-08-09,http://www.accuweather.com/en/us/glendale-az/8...,1,False,34,False,NaN,NaN,No,0
4,2020-08-10,http://www.accuweather.com/en/us/glendale-az/8...,2,False,34,False,NaN,NaN,No,0
...,...,...,...,...,...,...,...,...,...,...
1175,2020-08-11,http://www.accuweather.com/en/us/loveland-co/8...,4,False,34,False,NaN,NaN,No,0
1176,2020-08-12,http://www.accuweather.com/en/us/loveland-co/8...,3,False,34,False,NaN,NaN,No,0
1177,2020-08-13,http://www.accuweather.com/en/us/loveland-co/8...,3,False,33,False,NaN,NaN,No,0
1178,2020-08-14,http://www.accuweather.com/en/us/loveland-co/8...,4,False,34,False,NaN,NaN,No,0


In [28]:
# Based on the combination of 'Thunder' and 'Intensity' columns, we will assign a color code. If no thunder(green),
# thunder of light or moderate intensity (yellow), thunder of heavy intensity (red).

# Create a function
def c(row):
    if row['Thunder'] == 'Yes' and row['Intensity'] == 1:
        valc = 'Yellow'
    elif row['Thunder'] == 'Yes' and row['Intensity'] == 2:
        valc = 'Yellow'
    elif row['Thunder'] == 'Yes' and row['Intensity'] == 3: 
        valc = 'Red'
    else:
        valc = 'Green'
    return valc

fc_venue['Color_Code'] = fc_venue.apply(c, axis=1)

fc_venue

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


,Date,Link,Day_Icon,Day_HasPrecip,Night_Icon,Night_HasPrecip,Day_Intensity,Night_Intensity,Thunder,Intensity,Color_Code
0,2020-08-06,http://www.accuweather.com/en/us/glendale-az/8...,1,False,34,False,NaN,NaN,No,0,Green
1,2020-08-07,http://www.accuweather.com/en/us/glendale-az/8...,1,False,33,False,NaN,NaN,No,0,Green
2,2020-08-08,http://www.accuweather.com/en/us/glendale-az/8...,1,False,33,False,NaN,NaN,No,0,Green
3,2020-08-09,http://www.accuweather.com/en/us/glendale-az/8...,1,False,34,False,NaN,NaN,No,0,Green
4,2020-08-10,http://www.accuweather.com/en/us/glendale-az/8...,2,False,34,False,NaN,NaN,No,0,Green
...,...,...,...,...,...,...,...,...,...,...,...
1175,2020-08-11,http://www.accuweather.com/en/us/loveland-co/8...,4,False,34,False,NaN,NaN,No,0,Green
1176,2020-08-12,http://www.accuweather.com/en/us/loveland-co/8...,3,False,34,False,NaN,NaN,No,0,Green
1177,2020-08-13,http://www.accuweather.com/en/us/loveland-co/8...,3,False,33,False,NaN,NaN,No,0,Green
1178,2020-08-14,http://www.accuweather.com/en/us/loveland-co/8...,4,False,34,False,NaN,NaN,No,0,Green


## Step 9: Create a single data frame with venue location and weather forecast information

In [29]:
# So, we have 3 dataframes that we need to merge into a single dataframe
# The dataframes 'venues' and 'fc_df' both have a column that contains the postal code. This should be a simple
# merge, although we will have to change the column name in one of the dataframes so they have the same name.

# Let's change the 'PrimaryPostalCode' column name in the 'fc_df' dataframe to 'Zip_Code' to match the 'venues' dataframe
fc_df.columns = ['Key', 'Zip_Code', 'Country']
fc_df

,Key,Zip_Code,Country
0,36718_PC,85301,US
1,36549_PC,85035,US
2,36719_PC,85302,US
3,36714_PC,85296,US
4,36626_PC,85201,US
...,...,...,...
113,34424_PC,78664,US
114,35208_PC,80465,US
115,35083_PC,80203,US
116,35047_PC,80111,US


In [30]:
# Check the column names of 'venues' dataframe
venues.columns

Index(['Venue_Name', 'City', 'State', 'Zip_Code', 'Latitude', 'Longitude'], dtype='object')

In [31]:
# Merge the two dataframes
venues_new = venues.merge(fc_df, how='inner')

In [32]:
# Check the new dataframe to make sure it worked as intended
venues_new

,Venue_Name,City,State,Zip_Code,Latitude,Longitude,Key,Country
0,E. Lowell Rogers Amphitheatre,Glendale,AZ,85301,33.538792,-112.185368,36718_PC,US
1,Westside Amphitheatre,Phoenix,AZ,85035,33.472633,-112.234703,36549_PC,US
2,Soleri Amphitheatre,Glendale,AZ,85302,33.567440,-112.188331,36719_PC,US
3,Amphitheatre Park,Gilbert,AZ,85296,33.344061,-111.699973,36714_PC,US
4,Mesa Amphitheatre,Mesa,AZ,85201,33.425680,-111.837444,36626_PC,US
...,...,...,...,...,...,...,...,...
112,Round Rock Amphitheatre,Round Rock,TX,78664,30.506010,-97.681377,34424_PC,US
113,Red Rocks Park & Amphitheatre,Morrison,CO,80465,39.665380,-105.205861,35208_PC,US
114,City Hall Amphitheatre,Denver,CO,80203,39.734710,-104.987195,35083_PC,US
115,Fiddler's Green Amphitheatre,Greenwood Village,CO,80111,39.600112,-104.893555,35047_PC,US


In [33]:
# Looks good! Now comes the tricky part...the venues weather forecast dataframe does not contain columns
# that are similar to our venues_new dataframe. The idea is to create a single dataframe that has all of the 
# venue location and weather forecast information to use for plotting on a map.

# The venues forecast df does have a website link with the location key contained in the url. We will have to parse that out
# and add it to a column we will create and name 'Key' in the venues forecast df
# There is a python function for parsing url's called 'urlparse'. Let's use that!

from urllib.parse import urlparse # Import the function

# Now we need to create a loop to get the location key from each link in our fc_venue df
keys = []

for c in range(len(fc_venue)):
    u_link = fc_venue['Link'][c]
    parse_url = urlparse(u_link)
    path = parse_url.path
    locat_key = path.rpartition('/')[2]
    keys.append(locat_key)

In [34]:
# Create a dataframe with the location keys
keysdf = pd.DataFrame(data = keys, columns = ['Key'])

In [35]:
# Convert to upper case since we need to merge keysdf and venues_new
keysdf = keysdf.apply(lambda x: x.astype(str).str.upper())
keysdf

,Key
0,36718_PC
1,36718_PC
2,36718_PC
3,36718_PC
4,36718_PC
...,...
1175,35258_PC
1176,35258_PC
1177,35258_PC
1178,35258_PC


In [36]:
# Add the location key column to the fc_venue df
fc_venue = pd.concat([fc_venue, keysdf], axis=1)

fc_venue

,Date,Link,Day_Icon,Day_HasPrecip,Night_Icon,Night_HasPrecip,Day_Intensity,Night_Intensity,Thunder,Intensity,Color_Code,Key
0,2020-08-06,http://www.accuweather.com/en/us/glendale-az/8...,1,False,34,False,NaN,NaN,No,0,Green,36718_PC
1,2020-08-07,http://www.accuweather.com/en/us/glendale-az/8...,1,False,33,False,NaN,NaN,No,0,Green,36718_PC
2,2020-08-08,http://www.accuweather.com/en/us/glendale-az/8...,1,False,33,False,NaN,NaN,No,0,Green,36718_PC
3,2020-08-09,http://www.accuweather.com/en/us/glendale-az/8...,1,False,34,False,NaN,NaN,No,0,Green,36718_PC
4,2020-08-10,http://www.accuweather.com/en/us/glendale-az/8...,2,False,34,False,NaN,NaN,No,0,Green,36718_PC
...,...,...,...,...,...,...,...,...,...,...,...,...
1175,2020-08-11,http://www.accuweather.com/en/us/loveland-co/8...,4,False,34,False,NaN,NaN,No,0,Green,35258_PC
1176,2020-08-12,http://www.accuweather.com/en/us/loveland-co/8...,3,False,34,False,NaN,NaN,No,0,Green,35258_PC
1177,2020-08-13,http://www.accuweather.com/en/us/loveland-co/8...,3,False,33,False,NaN,NaN,No,0,Green,35258_PC
1178,2020-08-14,http://www.accuweather.com/en/us/loveland-co/8...,4,False,34,False,NaN,NaN,No,0,Green,35258_PC


In [37]:
# We only need to keep a few columns

fc_venue = fc_venue[['Date', 'Color_Code', 'Key']]
fc_venue.head()

,Date,Color_Code,Key
0,2020-08-06,Green,36718_PC
1,2020-08-07,Green,36718_PC
2,2020-08-08,Green,36718_PC
3,2020-08-09,Green,36718_PC
4,2020-08-10,Green,36718_PC


In [38]:
fc_venue_one = fc_venue.pivot(index = 'Key', columns = 'Date')
fc_venue_one

Color_Code                                                         \
Date     2020-08-06 2020-08-07 2020-08-08 2020-08-09 2020-08-10 2020-08-11   
Key                                                                          
1053_PC       Green        Red     Yellow      Green      Green      Green   
10834_PC      Green      Green      Green      Green     Yellow      Green   
11209_PC        Red        Red     Yellow      Green      Green     Yellow   
11339_PC        Red        Red     Yellow      Green      Green      Green   
11387_PC        Red     Yellow     Yellow     Yellow      Green      Green   
...             ...        ...        ...        ...        ...        ...   
8843_PC         Red        Red     Yellow      Green      Green      Green   
9010_PC         Red        Red     Yellow      Green      Green     Yellow   
9137_PC         Red        Red     Yellow      Green      Green      Green   
946_PC        Green      Green     Yellow      Green      Green      Green   
9599_PC         Red     Yellow     Yellow      Green      Green      Green   

                                                      
Date     2020-08-12 2020-08-13 2020-08-14 2020-08-15  
Key                                                   
1053_PC       Green      Green      Green      Green  
10834_PC      Green      Green      Green      Green  
11209_PC      Green      Green      Green      Green  
11339_PC      Green      Green      Green      Green  
11387_PC      Green      Green      Green      Green  
...             ...        ...        ...        ...  
8843_PC       Green      Green      Green      Green  
9010_PC       Green      Green      Green      Green  
9137_PC       Green      Green      Green      Green  
946_PC        Green      Green      Green      Green  
9599_PC       Green      Green      Green      Green  

[118 rows x 10 columns]

In [39]:
venue_forecasts_one = pd.merge(fc_venue_one, venues_new, left_index=True, right_on='Key')
venue_forecasts_one

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/pandas/core/reshape/merge.py:643: UserWarning: merging between different levels can give an unintended result (2 levels on the left,1 on the right)
  warnings.warn(msg, UserWarning)


,"(Color_Code, 2020-08-06)","(Color_Code, 2020-08-07)","(Color_Code, 2020-08-08)","(Color_Code, 2020-08-09)","(Color_Code, 2020-08-10)","(Color_Code, 2020-08-11)","(Color_Code, 2020-08-12)","(Color_Code, 2020-08-13)","(Color_Code, 2020-08-14)","(Color_Code, 2020-08-15)",Venue_Name,City,State,Zip_Code,Latitude,Longitude,Key,Country
103,Green,Red,Yellow,Green,Green,Green,Green,Green,Green,Green,J. Harold Williams Amphitheatre,Rockville,RI,02873,41.518690,-71.776710,1053_PC,US
87,Green,Green,Green,Green,Yellow,Green,Green,Green,Green,Green,Dr. Fred H. Tilock Amphitheatre,Bethany,WV,26032,40.205883,-80.560535,10834_PC,US
34,Red,Red,Yellow,Green,Green,Yellow,Green,Green,Green,Green,Amphitheatre At The Historic Depot,Burlington,NC,27215,36.093891,-79.436406,11209_PC,US
32,Red,Red,Yellow,Green,Green,Green,Green,Green,Green,Green,Koka Booth Amphitheatre,Cary,NC,27518,35.728888,-78.795916,11339_PC,US
33,Red,Yellow,Yellow,Yellow,Green,Green,Green,Green,Green,Green,Neuse River Amphitheatre,Smithfield,NC,27577,35.507064,-78.354228,11387_PC,US
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,Red,Red,Yellow,Green,Green,Green,Green,Green,Green,Green,Baltimore Inner Harbor Amphitheatre,Baltimore,MD,21201,39.285904,-76.612120,8843_PC,US
73,Red,Red,Yellow,Green,Green,Yellow,Green,Green,Green,Green,Carroll Creek Park Amphitheatre,Frederick,MD,21701,39.412654,-77.408960,9010_PC,US
69,Red,Red,Yellow,Green,Green,Green,Green,Green,Green,Green,Mason District Park Amphitheatre,Annandale,VA,22003,38.834846,-77.174332,9137_PC,US
85,Green,Green,Yellow,Green,Green,Green,Green,Green,Green,Green,Province Lands Amphitheatre,Provincetown,MA,02657,42.074059,-70.205307,946_PC,US


In [40]:
venue_forecasts_one.columns

Index([('Color_Code', '2020-08-06'), ('Color_Code', '2020-08-07'),
       ('Color_Code', '2020-08-08'), ('Color_Code', '2020-08-09'),
       ('Color_Code', '2020-08-10'), ('Color_Code', '2020-08-11'),
       ('Color_Code', '2020-08-12'), ('Color_Code', '2020-08-13'),
       ('Color_Code', '2020-08-14'), ('Color_Code', '2020-08-15'),
                       'Venue_Name',                       'City',
                            'State',                   'Zip_Code',
                         'Latitude',                  'Longitude',
                              'Key',                    'Country'],
      dtype='object')

## Step 10: Plot venue locations with the 10-day forecast. Green = no thunderstorms, Yellow = thunderstorms expected but no risk for high wind, hail or tornadoes, Red = thunderstorms expected with a risk on at least 1 day for high wind, hail or tornadoes.

In [43]:
# create map
outdoor_venues = folium.Map(location=[39.828329, -98.579478], zoom_start=4)

# add markers to the map
markers_colors = []
for lat, lon, poi, day1, day2, day3, day4, day5, day6, day7, day8, day9, day10 in zip(venue_forecasts_one['Latitude'], 
                                                                                      venue_forecasts_one['Longitude'], 
                                                                                      venue_forecasts_one['Venue_Name'],
                                                                                      venue_forecasts_one[('Color_Code', '2020-08-06')],
                                                                                      venue_forecasts_one[('Color_Code', '2020-08-07')],
                                                                                      venue_forecasts_one[('Color_Code', '2020-08-08')],
                                                                                      venue_forecasts_one[('Color_Code', '2020-08-09')],
                                                                                      venue_forecasts_one[('Color_Code', '2020-08-10')],
                                                                                      venue_forecasts_one[('Color_Code', '2020-08-11')],
                                                                                      venue_forecasts_one[('Color_Code', '2020-08-12')],
                                                                                      venue_forecasts_one[('Color_Code', '2020-08-13')],
                                                                                      venue_forecasts_one[('Color_Code', '2020-08-14')],
                                                                                      venue_forecasts_one[('Color_Code', '2020-08-15')]
                                                                                     ):
    label = folium.Popup('Venue: ' + str(poi) +
                         ', Day1: ' + str(day1) + 
                         ', Day2: ' + str(day2) + 
                         ', Day3: ' + str(day3) +  
                         ', Day4: ' + str(day4) + 
                         ', Day5: ' + str(day5) + 
                         ', Day6: ' + str(day6) + 
                         ', Day7: ' + str(day7) + 
                         ', Day8: ' + str(day8) + 
                         ', Day9: ' + str(day9) + 
                         ', Day10: ' + str(day10), parse_html = True, max_width = 250)
                         
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='black',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7).add_to(outdoor_venues)
       
outdoor_venues